# Republican Caucus Miscellaneous Measurements

This notebook contains some miscellaneous calculations added to our paper during the review process. Among other things, it calculates the Holding Times found in Table 5.

To run this notebook you'll first need to run the Trader Analysis notebooks first.

In [1]:
import os
import sys
import pickle

from collections import defaultdict

import numpy as np
import pandas as pd
from pandas import Series

from research_tools import storage

# Load Data

In [2]:
os.chdir('..')

basename = 'gop'

trader_analysis, = storage.retrieve_all([basename + '.trader_analysis'])

Reading data from data/gop.trader_analysis.p


In [3]:
trader_analysis.head()

index            seq  market_id  contract_id                    date_executed                             user_guid trade_type  price_per_share  \
contract_id user_guid                                                                                                                                                                               
616         0026E24F-32BF-386C-A133-9C0061E04278   4712  1442559530907       1327          616 2015-09-18 02:58:50.907000-04:00  0026E24F-32BF-386C-A133-9C0061E04278    Buy Yes             0.19   
            0026E24F-32BF-386C-A133-9C0061E04278   8063  1444379167587       1327          616 2015-10-09 04:26:07.587000-04:00  0026E24F-32BF-386C-A133-9C0061E04278   Sell Yes             0.13   
            0055E87B-7662-3DC4-934F-144884375093    470  1426965643440       1327          616 2015-03-21 15:20:43.440000-04:00  0055E87B-7662-3DC4-934F-144884375093     Buy No             0.79   
            0055E87B-7662-3DC4-934F-144884375093    472  1427175738867       1327          616 2015-03-24 01:42:18.867000-04:00  0055E87B-7662-3DC4-934F-144884375093     Buy No             0.81   
            0055E87B-7662-3DC4-934F-144884375093    473  1427520569703       1327          616 2015-03-28 01:29:29.703000-04:00  0055E87B-7662-3DC4-934F-144884375093     Buy No             0.84   

                                                  quantity  placed_order_id     ...       corrected_side  take_provide  notional buy_sell  cash_flow  yes_no  gross_pnl  fee  pnl_net_fee  close_trade  
contract_id user_guid                                                           ...                                                                                                                     
616         0026E24F-32BF-386C-A133-9C0061E04278         1           217527     ...                    1             T      0.19        1      -0.19     Yes       0.00  0.0         0.00        False  
            0026E24F-32BF-386C-A133-9C0061E04278         1           301314     ...                   -1             T      0.13       -1       0.13     Yes      -0.06  0.0        -0.06        False  
            0055E87B-7662-3DC4-934F-144884375093         1            56230     ...                   -1             T      0.79        1      -0.79      No       0.00  0.0         0.00        False  
            0055E87B-7662-3DC4-934F-144884375093         6            56268     ...                   -1             T      4.86        1      -4.86      No       0.00  0.0         0.00        False  
            0055E87B-7662-3DC4-934F-144884375093        10            56268     ...                   -1             P      8.40        1      -8.40      No       0.00  0.0         0.00        False  

[5 rows x 22 columns]

# Traders with Positions at Market Close

How many traders had a position at market close?

In [4]:
close_traders = trader_analysis.groupby('user_guid').apply(lambda x: x.close_trade.any())

close_traders.head()

user_guid
000DA83C-D191-3726-A4D5-4E56F0CC7F80    True
00140D47-5573-38BE-8E79-AA60B9D8563D    True
001AE9D0-FC38-3273-9DB1-973C0678E270    True
0026E24F-32BF-386C-A133-9C0061E04278    True
0055E87B-7662-3DC4-934F-144884375093    True
dtype: bool

In [5]:
close_traders.sum() / close_traders.count()

0.72304582210242585

In [6]:
close_traders.sum(), close_traders.count()

(3219, 4452)

# Average Holding Time

What is the average holding time of positions? And median holding time?

In [7]:
holding_times = []
position_open_timestamps = defaultdict(list)

for trade in trader_analysis.itertuples():
    if trade.buy_sell == 1:
        position_open_timestamps[(trade.contract_id, trade.user_guid)].extend([trade.seq] * trade.quantity)
    elif trade.buy_sell == -1:
        oldest_timestamps = position_open_timestamps[(trade.contract_id, trade.user_guid)][:trade.quantity]
        for t in oldest_timestamps:
            holding_times.append(trade.seq - t)
        position_open_timestamps[(trade.contract_id, trade.user_guid)] = (
            position_open_timestamps[(trade.contract_id, trade.user_guid)][trade.quantity:]
        )

holding_times = Series(holding_times)

In [8]:
pd.Timedelta(holding_times.mean(), 'ms')

Timedelta('16 days 06:10:58.401948')

In [9]:
holding_times.mean() / (24 * 60 * 60 * 1000)

16.257620392916067

In [10]:
pd.Timedelta(holding_times.quantile(0.5), 'ms')

Timedelta('4 days 17:37:18.540000')

In [11]:
holding_times.quantile(0.5) / (24 * 60 * 60 * 1000)

4.734242361111111

Save the holding times data to a pickle file so we can combine it with the data from the other market.

In [12]:
with open('data/gop.holding_times.p', 'wb') as f:
    pickle.dump(holding_times, f)

# Pre-Margin Linking Volume

How much volume was during the pre-margin linking period?

In [13]:
rep_margin_linking_cutoff_date = '2015-12-01'
pre_margin_linking = trader_analysis.date_executed < rep_margin_linking_cutoff_date

trader_analysis[pre_margin_linking].quantity.sum() / trader_analysis.quantity.sum()

0.08303207694877561